## SQLite3 and Python Types

SQLite supports the following types: NULL, INTEGER, REAL, TEXT, BLOB (block object). Thus, the following Python types can work with SQLite without any problem:

| Python type           | SQLite type  |
|-----------------------|--------------|
| None                  | NULL         |
| int                   | INTEGER      |
| float                 | Real         |
| str                   | TEXT         |
| bytes                 | BLOB         |


SQLite types are converted to Python types by  default in the following way:

| SQLite type | Python type  |
|-------------|--------------|
| NULL        | None         |
| INTEGER     | int          |
| REAL        | float        |
| TEXT        | str (default)|
| BLOB        | bytes        |

## Store additional Python types in SQLite databases

SQLite database supports only a limited set of types natively but can be extended with additional Python types. There are two ways to use other Python types with SQLite:

1. **Adaptation**
2. **Converters**

### Adaptation

If you have a Python type that is not supported by SQLite, you can adapt it to one of these sqlite3 module's supported types:

- None
- int
- long
- float
- str
- unicode
- buffer

There are 2 ways to do adaptation:

### 1. Let your object adapt itself

This way is good if you write the class yourself. Let's revisit the class Point with x and y coordinates.

In [6]:
class Point():
    def __init__(self, x, y):
        self.x, self.y = x, y

Suppose you want to store a Point object in a single SQLite column. To represent the point we need to use one of the supported types listed above. Let's just use str type and separate the coordinates using a semicolon. Then you need to give your class a method \__conform\__(self, protocol) which must return the converted value. The parameter protocol will be **PrepareProtocol**.

In [10]:
import sqlite3

class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    # returns a Point object as a string
    def __conform__(self, protocol):
        if protocol is sqlite3.PrepareProtocol:
            return "%d;%d" %(self.x, self.y)

# create a database in memory (RAM)
con = sqlite3.connect(":memory:")
cur = con.cursor()

p = Point(4.0, -3.2)
cur.execute("select ?", (p,))
print(cur.fetchone()[0])

('4;-3',)


### 2. Register the adapter

The second way is to create a function that converts the object to the string representation and register the function with register_adapter().

In [15]:
class Point():
    def __init__(self, x, y):
        self.x, self.y = x, y

def adapt_point(self):
    return "%d;%d" % (self.x, self.y)

# register function adapt_point
sqlite3.register_adapter(Point, adapt_point)

con = sqlite3.connect(":memory:")
cur = con.cursor()

p = Point(4.0, -3.2)
cur.execute("select ?", (p,))
print(cur.fetchone()[0])

4;-3


## Converters

### Converting SQLite values to custom Python types

We saw how to save objects of custom Python types in SQLite database. However, it is more useful to make the round trip work, that is, **Python-SQLite-Python** conversions. **Here comes the role of converters**.

Going back to the Point class. We stored the x and y coordinates separated via semicolons as strings in SQLite column.

To control how database types are mapped into Python objects when results are fetched from the database server, we create a  a **converter function**. This function accepts the string representing the point as a parameter and constructs a Point object from it.

**NOTE**: Converter functions always get called with a bytes object, no matter under which data type you sent the value to SQLite.

In [8]:
def convert_point(s):
    x, y = map(float, s.split(b";"))
    return Point(x, y)

Now you need to make the sqlite3 module know that what you have selected from the database is actually a point. There are two ways of doing that:

#### A. Implicitly via the declared type  
- $\rightarrow$ using the constant PARSE_DECLTYPES


#### B. Explicitly via the column name 
- $\rightarrow$ using the constant PARSE_COLNAMES


The following example illustrates both ways.

In [11]:
import sqlite3

class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __repr__(self):
        return "(%f;%f)" % (self.x, self.y)

def adapt_point(p):
    return "%f;%f" % (p.x, p.y)

def convert_point(s):
    x, y = map(float, s.split(b";"))
    return Point(x, y)

# register the adapter function 
# from the Python type (class) Point 
# to any Python types handled directly by sqlite3 such as int, float, or str
sqlite3.register_adapter(Point, adapt_point)

# register the converter function as being of a type point
sqlite3.register_converter("point", convert_point)

p = Point(-4.0, 2.3)

######### (A) Declared Types #############

# the constant PARSE_DECLTYPES is used with the detect_types parameter of the connect() function
# makes sqlite3 module parse the DECLARED TYPE for each column it returns
# then for that column, it will look into the convertors dictionary and then
# use the converter function registered for that type there
# NOTE that converter names are case-sensitive!
con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()

cur.execute("create table test(p point)")

cur.execute("insert into test(p) values (?)", (p,))
cur.execute("select p from test")
print("with declared types:", cur.fetchone()[0])
cur.close()
con.close()

######### (B) Column Names #############
# the constant PARSE_COLNAMES is used with the detect_types parameter of the connect function
# makes the SQLite3 parse the COLUMN NAME for each column it returns
# it will look for a string formed [mytype] in there, and then decide that 'mytype' is the type of the column
# it will try to find an entry of 'mytype' in the converters dictionary and 
# then use the converter function found there to return the value. 
con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_COLNAMES)
cur = con.cursor()

cur.execute("create table test(p)")
cur.execute("insert into test(p) values (?)", (p,))
cur.execute('select p as "p [point]" from test')
print("with column names:", cur.fetchone()[0])
cur.close()
con.close()

with declared types: (-4.000000;2.300000)
with column names: (-4.000000;2.300000)


## Perfect!

### You have MASTERED how to handle custom Python data types and incorporate them in SQLite databases. Next, you'll learn how to use the SQLite databases in an efficient way.